# GCMC Cornac experiments
### 1. Init
#### 1.1. Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
sys.path.append("../../")


import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from DBPR import utils
utils.set_seed(0)
from DBPR import dataset
from DBPR import model
from experiments.datasets.data_utils import cornac_util

import optuna
import gc
import json
import torch
import pandas as pd
from importlib import reload


from cornac.models import PMF
from cornac.metrics import MAE, RMSE, RatingMetric
import logging
import cornac

#### 1.2. Set up the loggers

In [ ]:
utils.setuplogger(verbose = True, log_name="GCMC_cornac")

#### 1.4. Parametrize the datasets

In [ ]:
# choose dataset here
dataset_name = 'postcovid'
version= ""#"_small"
# modify config here

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

config = {

    # General params
    'seed' : 0,

    # Saving params
    'load_params': False,
    'save_params': False,
    'embs_path' : '../embs/'+str(dataset_name),
    'params_path' :'../ckpt/'+str(dataset_name),

    # training mode
    'early_stopping' : True,
    'fast_training' : True, # (Only taken in account if early_stopping == true) If true, doesn't compute valid rmse PC-ER

    # Learning params
    'learning_rate': 0.001,
    'batch_size': 2048,
    'num_epochs': 200,
    'num_dim': 10, # for IRT or MIRT todo : is it necessary as we use concepts knowledge number as embedding dimension ?
    'eval_freq' : 1,
    'patience' : 30,
    'device': device,
    'lambda' : 7.7e-6,
    'tensorboard': False,
    'flush_freq' : True,

    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
    'best_params_path':'',

    #For GCCD
    'num_layers': 0,
    'version': 'pair',
    'p_dropout': 0,
    'low_mem_mode' : True,
    'user_nbrs_n' : 10,
    'item_nbrs_n' : 5
}
concept_map = json.load(open(f'../datasets/{dataset_name}/concept_map.json', 'r'))
concept_map = {int(k):[int(x) for x in v] for k,v in concept_map.items()}
metadata = json.load(open(f'../datasets/{dataset_name}/metadata.json', 'r'))
utils.set_seed(config['seed'])
dataset_name += version
logging.info(f'#### {dataset_name} ####')
logging.info(f'#### config : {config} ####')

In [ ]:
def generate_GCMC(config,metadata = None) :
    return cornac.models.GCMC(
        max_iter=config['num_epochs'],
        learning_rate=config['learning_rate'],
        gcn_out_units=metadata['num_dimension_id'],
        optimizer='adam',
        gcn_agg_accum="sum",
        activation_func=config['activation_func'],
        gcn_agg_units=500,
        train_valid_interval=config['eval_freq'],
        train_early_stopping_patience=config['patience'],
        trainable=True,
        seed=config['seed'],
        verbose=False
    )

### 2. CDM Training


In [ ]:
reload(utils)
reload(model)
reload(dataset)

seed = 0
utils.set_seed(0)

config['seed'] = seed
config['early_stopping'] = True
config['esc'] = 'objectives' #'loss' 'delta_objectives'
config['num_epochs']=200
config['eval_freq']=1
config['patience']=30

config['verbose_early_stopping'] = False
config["tensorboard"] = False
config['flush_freq'] = False
config['save_params']= False
config['disable_tqdm'] = True

In [ ]:
reload(cornac_util)

dataset_name = "postcovid"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_GCMC(trial, config, eval_method,generate_GCMC), n_trials=100, timeout=1800, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
pareto_trials = study.best_trials
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")

dataset_name = "promis"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_GCMC(trial, config, eval_method,generate_GCMC), n_trials=100, timeout=3600, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")

dataset_name = "movielens"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_GCMC(trial, config, eval_method,generate_GCMC), n_trials=100, timeout=3600, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")
    
dataset_name = "portrait"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_GCMC(trial, config, eval_method,generate_GCMC), n_trials=100, timeout=3600, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")


### 3. CDM Prediction
#### 3.1. Training and testing

In [ ]:
def find_emb(algo):
    enc_graph = algo.model.train_enc_graph
    emb, item_out =  algo.model.net.encoder(enc_graph)
    return emb.detach().cpu().numpy()

In [9]:
reload(cornac_util)

# dataset_name = "postcovid"
# logging.info(dataset_name)
# config['learning_rate'] = 0.01415
# config['activation_func'] = 'tanh'
# metrics = cornac_util.test(dataset_name,config,generate_GCMC,find_emb)

# dataset_name = "promis"
# logging.info(dataset_name)
# config['learning_rate'] = 0.02032
# config['activation_func'] = 'relu'
# metrics = cornac_util.test(dataset_name,config,generate_GCMC,find_emb)
#
# dataset_name = "movielens"
# logging.info(dataset_name)
# config['learning_rate'] = 0.01892
# config['activation_func'] = 'tanh'
# metrics = cornac_util.test(dataset_name,config,generate_GCMC,find_emb)
#
dataset_name = "portrait"
logging.info(dataset_name)
config['learning_rate'] = 0.01541
config['activation_func'] = 'tanh'
metrics = cornac_util.test(dataset_name,config,generate_GCMC,find_emb)

[INFO 18:07] portrait
[INFO 18:09] Train enc graph: 1152 users, 116 items, 79488 pairs
[INFO 18:09] Train dec graph: 1152 users, 116 items, 79488 pairs
[INFO 18:09] Valid enc graph: 1152 users, 116 items, 79488 pairs
[INFO 18:09] Valid dec graph: 1152 users, 116 items, 26496 pairs
[INFO 18:09] Training Started!


Training:   0%|          | 0/199 [00:00<?, ?iter/s]

[INFO 18:09] Total # params of net: 65311386
[INFO 18:09] Total Param Number: 65311386
Params:
	encoder.ufc.weight: torch.Size([9, 500]), 4500
	encoder.ufc.bias: torch.Size([9]), 9
	encoder.ifc.weight: torch.Size([9, 500]), 4500
	encoder.ifc.bias: torch.Size([9]), 9
	encoder.conv.mods.1_0.weight: torch.Size([1152, 500]), 576000
	encoder.conv.mods.rev-1_0.weight: torch.Size([116, 500]), 58000
	encoder.conv.mods.1_01.weight: torch.Size([1152, 500]), 576000
	encoder.conv.mods.rev-1_01.weight: torch.Size([116, 500]), 58000
	encoder.conv.mods.1_02.weight: torch.Size([1152, 500]), 576000
	encoder.conv.mods.rev-1_02.weight: torch.Size([116, 500]), 58000
	encoder.conv.mods.1_03.weight: torch.Size([1152, 500]), 576000
	encoder.conv.mods.rev-1_03.weight: torch.Size([116, 500]), 58000
	encoder.conv.mods.1_04.weight: torch.Size([1152, 500]), 576000
	encoder.conv.mods.rev-1_04.weight: torch.Size([116, 500]), 58000
	encoder.conv.mods.1_05.weight: torch.Size([1152, 500]), 576000
	encoder.conv.mods.re

Training:   1%|          | 1/199 [00:00<00:29,  6.81iter/s]

[INFO 18:09] Epoch: 2	loss: 4.6347	rmse:0.1249	time:nan	Val rmse=0.3530


...

[INFO 18:51] Epoch: 32	loss: 4.1863	rmse:0.0809	time:0.0862	Val rmse=0.2795


Training:  16%|█▌        | 32/199 [00:04<00:23,  6.97iter/s]

[INFO 18:51] Epoch: 33	loss: 4.1782	rmse:0.0800	time:0.0860	


Training:  17%|█▋        | 33/199 [00:04<00:24,  6.73iter/s]


KeyboardInterrupt: 